In [1]:
# Install any packages needed for model training
%pip install mlb-statsapi

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Set API Key from SportRadar and import needed libraries for data gathering
import pandas as pd
import time
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import mysql.connector
import statsapi

load_dotenv()

db_password = os.getenv('DB_PASSWORD')

# Load SQL database created in mySQL (SQL code provided in "sql" folder)
db = mysql.connector.connect(
    host="localhost",
    user="dgiletto",
    password=db_password,
    database="mlb"
)

cursor = db.cursor()

In [3]:
def get_games_on_date(date_str: str):
    """
    Get scheduled MLB games for a certain date using mlbstatsapi
    Parameters:
        date_str (str): A string representation of the date in the format (YYYY-MM-DD)
    Returns:
        List containing a dict for each game on the day
    """
    games = statsapi.schedule(date=date_str)
    return games

get_games_on_date("2025-06-14")

[{'game_id': 777514,
  'game_datetime': '2025-06-14T17:05:00Z',
  'game_date': '2025-06-14',
  'game_type': 'R',
  'status': 'Final',
  'away_name': 'Miami Marlins',
  'home_name': 'Washington Nationals',
  'away_id': 146,
  'home_id': 120,
  'doubleheader': 'N',
  'game_num': 1,
  'home_probable_pitcher': 'Trevor Williams',
  'away_probable_pitcher': 'Cade Gibson',
  'home_pitcher_note': '',
  'away_pitcher_note': '',
  'away_score': 4,
  'home_score': 3,
  'current_inning': 9,
  'inning_state': 'Bottom',
  'venue_id': 3309,
  'venue_name': 'Nationals Park',
  'national_broadcasts': ['MLBN (out-of-market only)'],
  'series_status': 'MIA wins 2-0',
  'winning_team': 'Miami Marlins',
  'losing_team': 'Washington Nationals',
  'winning_pitcher': 'Janson Junk',
  'losing_pitcher': 'Trevor Williams',
  'save_pitcher': 'Calvin Faucher',
  'summary': '2025-06-14 - Miami Marlins (4) @ Washington Nationals (3) (Final)'},
 {'game_id': 777517,
  'game_datetime': '2025-06-14T17:10:00Z',
  'game_d

In [4]:
def get_total_runs(game_id: str):
    """
    Get total runs scored for a particular game
    Parameters:
        game_id (str): ID which identifies a unique game
    Returns:
        The total amount of runs scored between the home and away team
    """
    try:
        box = statsapi.boxscore_data(gamePk=game_id)
        home_runs = box['home']['teamStats']['batting']['runs']
        away_runs = box['away']['teamStats']['batting']['runs']
        return home_runs + away_runs
    except:
        return None

get_total_runs("777245")

13

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time

def get_total_lines(date_str):
    url = f"https://www.sportsbookreview.com/betting-odds/mlb-baseball/totals/full-game/?date={date_str}"
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-gpu")
    driver = webdriver.Chrome(options=options)

    try:
        driver.get(url)

        # Wait up to 15 seconds for any span with "O" or "U" + number
        WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, "span"))
        )
        time.sleep(2)  # Add buffer for JavaScript to render fully

        # Now grab all bolds and spans and filter for total lines
        spans = driver.find_elements(By.TAG_NAME, "span")
        bolds = driver.find_elements(By.TAG_NAME, "b")
        totals = []
        pattern = re.compile(r'^[O]\s\d+(\.\d+)?$')

        for span in spans:
            text = span.text.strip()
            if pattern.match(text):
                totals.append({"home": "", "away": "", "line": text[2:]})
        
        totals_index = 0
        is_away = True
        # Use knowledge of <b> holding team abbreviations to access the matchup of the day
        for bold in bolds:
            text = bold.text.strip()
            if len(text) <= 3:
                if totals_index >= len(totals):
                    break  # Avoid IndexError
                
                if is_away:
                    totals[totals_index]["away"] = text
                else:
                    totals[totals_index]["home"] = text
                    totals_index += 1

                is_away = not is_away

        return totals

    finally:
        driver.quit()

get_total_lines("2025-07-08")

[{'home': 'BAL', 'away': 'NYM', 'line': '9.5'},
 {'home': 'DET', 'away': 'TB', 'line': '8'},
 {'home': 'NYY', 'away': 'SEA', 'line': '9'},
 {'home': 'CIN', 'away': 'MIA', 'line': '9'},
 {'home': 'BOS', 'away': 'COL', 'line': '9'},
 {'home': 'KC', 'away': 'PIT', 'line': '7.5'},
 {'home': 'CHW', 'away': 'TOR', 'line': '9'},
 {'home': 'MIL', 'away': 'LAD', 'line': '8.5'},
 {'home': 'MIN', 'away': 'CHC', 'line': '9.5'},
 {'home': 'STL', 'away': 'WAS', 'line': '8.5'},
 {'home': 'HOU', 'away': 'CLE', 'line': '7'},
 {'home': 'LAA', 'away': 'TEX', 'line': '7.5'},
 {'home': 'SD', 'away': 'AZ', 'line': '8.5'},
 {'home': 'SF', 'away': 'PHI', 'line': '10.5'}]

In [6]:
def abbreviation_conversion(abb):
    teams = {
        "AZ": "Arizona Diamondbacks",
        "ATL": "Atlanta Braves",
        "BAL": "Baltimore Orioles",
        "BOS": "Boston Red Sox",
        "CHW": "Chicago White Sox",
        "CHC": "Chicago Cubs",
        "CIN": "Cincinnati Reds",
        "CLE": "Cleveland Guardians",
        "COL": "Colorado Rockies",
        "DET": "Detroit Tigers",
        "HOU": "Houston Astros",
        "KC": "Kansas City Royals",
        "LAA": "Los Angeles Angels",
        "LAD": "Los Angeles Dodgers",
        "MIA": "Miami Marlins",
        "MIL": "Milwaukee Brewers",
        "MIN": "Minnesota Twins",
        "NYY": "New York Yankees",
        "NYM": "New York Mets",
        "ATH": "Athletics",
        "PHI": "Philadelphia Phillies",
        "PIT": "Pittsburgh Pirates",
        "SD": "San Diego Padres",
        "SF": "San Francisco Giants",
        "SEA": "Seattle Mariners",
        "STL": "St. Louis Cardinals",
        "TB": "Tampa Bay Rays",
        "TEX": "Texas Rangers",
        "TOR": "Toronto Blue Jays",
        "WAS": "Washington Nationals"
    }
    return teams.get(abb.upper(), "Unknown Team")

abbreviation_conversion("BOS")

'Boston Red Sox'

In [7]:
def calculate_ou_results(date_str):
    results = []
    totals = get_total_lines(date_str)

    for game in totals:
        home = abbreviation_conversion(game["home"])
        away = abbreviation_conversion(game["away"])
        line = float(game["line"])

        schedule = statsapi.schedule(start_date=date_str, end_date=date_str)
        game_id = None

        for g in schedule:
            if g["home_name"] == home and g["away_name"] == away:
                game_id = g["game_id"]
                break

        if not game_id:
            results.append({
                "game_id": 0,
                "home": home,
                "away": away,
                "line": line,
                "result": "Game not found"
            })
            continue
    
        total_runs = get_total_runs(game_id)

        if total_runs is None:
            result = "No result"
        elif total_runs > line:
            result = "Over"
        elif total_runs < line:
            result = "Under"
        else:
            result = "Push"

        results.append({
            "game_id": game_id,
            "home": home,
            "away": away,
            "line": line,
            "total_runs": total_runs,
            "result": result
        })
    
    return results

In [8]:
def get_team_stats(game_id, team_type='home'):
    """
    Fetch season stats for the given team type of the game inputted
    
    Parameters:
        game_id (str): ID which identifies a unique game
        team_type (str): 'home' or 'away' to specify which team's starter to get
    Returns:
        A dictionary which stores stats like batting average, era, obs, etc.
    """
    try:
        stats = statsapi.boxscore_data(gamePk=game_id)
        avg = stats[team_type]['teamStats']['batting']['avg']
        slg = stats[team_type]['teamStats']['batting']['slg']
        obp = stats[team_type]['teamStats']['batting']['obp']
        ops = stats[team_type]['teamStats']['batting']['ops']
        team_era = stats[team_type]['teamStats']['pitching']['era'] # Get bullpen data included
        pitcher_obp = stats[team_type]['teamStats']['pitching']['obp']

        return {
            'batting_avg': avg,
            'slg': slg,
            'obp': obp,
            'ops': ops,
            'team_era': team_era,
            'pitcher_obp': pitcher_obp
        }
    except:
        return None
    
get_team_stats("777245", team_type='away')

{'batting_avg': '.253',
 'slg': '.423',
 'obp': '.325',
 'ops': '.748',
 'team_era': '3.98',
 'pitcher_obp': '.257'}

In [22]:
def get_starting_pitcher_stats(pitcher_name):
    """
    Get stats for a pitcher by name using statsapi lookup
    
    Parameters:
        pitcher_name (str): Full name of the pitcher

    Returns:
        dict: Dictionary with pitcher name, ERA, WHIP, HR/9, and Ks/9
    """
    search = statsapi.lookup_player(pitcher_name)
    if not search:
        return {"error": f"Pitcher {pitcher_name} not found"}
    
    pitcher_id = 0

    for pitcher in search:
        if pitcher["fullName"] == pitcher_name and (pitcher["primaryPosition"]["code"] == '1' or pitcher["primaryPosition"]["abbreviation"] == 'TWP'):
            pitcher_id = pitcher['id']
            break
    try:
        stats = statsapi.player_stat_data(pitcher_id, group='pitching', type='season')
        pitching_stats = stats['stats'][0]
        return {
            "name": pitcher_name,
            "ERA": pitching_stats["stats"]["era"],
            "WHIP": pitching_stats["stats"]["whip"],
            "HR/9": pitching_stats["stats"]["homeRunsPer9"],
            "K/9": pitching_stats["stats"]["strikeoutsPer9Inn"],
            "R/9": pitching_stats["stats"]["runsScoredPer9"]
        }
    except Exception as e:
        return {"error": str(e)}

get_starting_pitcher_stats("Joe Musgrove")

{'error': 'Pitcher Joe Musgrove not found'}

In [10]:
def insert_game_record(db, cursor, record):
    try:
        query="""
        INSERT INTO mlb_game_data (
            game_date, home_team, away_team, 
            home_starter_era, away_starter_era, home_starter_whip, away_starter_whip,
            home_starter_hr, away_starter_hr, home_starter_k, away_starter_k,
            home_starter_r, away_starter_r, home_team_era, away_team_era,
            home_pitcher_obp, away_pitcher_obp, home_ba, away_ba, home_slg,
            away_slg, home_obp, away_obp, home_ops, away_ops, line, result
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = (
            record["date"],
            record["home_team"],
            record["away_team"],
            record["home_starter_era"],
            record["away_starter_era"],
            record["home_starter_whip"],
            record["away_starter_whip"],
            record["home_starter_HR/9"],
            record["away_starter_HR/9"],
            record["home_starter_K/9"],
            record["away_starter_K/9"],
            record["home_starter_R/9"],
            record["away_starter_R/9"],
            record["home_team_era"],
            record["away_team_era"],
            record["home_pitcher_obp"],
            record["away_pitcher_obp"],
            record["home_ba"],
            record["away_ba"],
            record["home_slg"],
            record["away_slg"],
            record["home_obp"],
            record["away_obp"],
            record["home_ops"],
            record["away_ops"],
            record["line"],
            record["result"]
        )
        cursor.execute(query, values)
        db.commit()
    except mysql.connector.Error as err:
        print("Insert failed: ", err)

In [14]:
from datetime import datetime, timedelta

# Assemble Dataset using a combination of our past functions
def collect_game_data(start_date, end_date):
    dataset = []
    current = start_date
    # Iterate throught each day of games until we get to the last day
    while current <= end_date:
        # Get all of the games on the current day
        games = get_games_on_date(current.strftime("%Y-%m-%d"))
        # Obtain our target
        lines = calculate_ou_results(current.strftime("%Y-%m-%d"))
        if not lines:
            continue
        
        for game in games:
            game_id = game["game_id"]
            line_data = next((line for line in lines if line["game_id"] == game_id), None)
            if not line_data:
                continue
            
            home_name = game["home_name"]
            away_name = game["away_name"]
            home_starter = game["home_probable_pitcher"]
            away_starter = game["away_probable_pitcher"]

            # Obtain our features
            home_stats = get_team_stats(game_id, team_type="home")
            away_stats = get_team_stats(game_id, team_type="away")
            if not home_stats or not away_stats:
                continue

            home_starter_stats = get_starting_pitcher_stats(home_starter)
            away_starter_stats = get_starting_pitcher_stats(away_starter)

            if (
                not home_starter_stats or "error" in home_starter_stats or
                not away_starter_stats or "error" in away_starter_stats
            ):
                print(f"Skipping game {game_id} due to missing pitcher stats: {home_starter_stats.get('error')}, {away_starter_stats.get('error')}")
                continue

            # Append it all into a dict
            record = {
                'date': current.strftime("%Y-%m-%d"),
                'home_team': home_name,
                'away_team': away_name,
                'home_starter_era': home_starter_stats["ERA"],
                'away_starter_era': away_starter_stats["ERA"],
                'home_starter_whip': home_starter_stats["WHIP"],
                'away_starter_whip': away_starter_stats["WHIP"],
                'home_starter_HR/9': home_starter_stats["HR/9"],
                'away_starter_HR/9': away_starter_stats["HR/9"],
                'home_starter_K/9': home_starter_stats["K/9"],
                'away_starter_K/9': away_starter_stats["K/9"],
                'home_starter_R/9': home_starter_stats["R/9"],
                'away_starter_R/9': away_starter_stats["R/9"],
                'home_team_era': home_stats["team_era"],
                'away_team_era': away_stats["team_era"],
                'home_pitcher_obp': home_stats["pitcher_obp"],
                'away_pitcher_obp': away_stats["pitcher_obp"],
                'home_ba': home_stats["batting_avg"],
                'away_ba': away_stats["batting_avg"],
                'home_slg': home_stats["slg"],
                'away_slg': away_stats["slg"],
                'home_obp': home_stats["obp"],
                'away_obp': away_stats["obp"],
                'home_ops': home_stats["ops"],
                'away_ops': away_stats["ops"],
                'line': line_data["line"],
                'result': line_data["result"]
            }

            insert_game_record(db, cursor, record)
            time.sleep(1) # to avoid wait limit
        current += timedelta(days=1)
    # Return a data frame with rows for each game
    return dataset

collect_game_data(
    start_date=datetime(2024, 9, 20),
    end_date=datetime(2024, 9, 30)
)

Skipping game 745364 due to missing pitcher stats: Pitcher Joe Musgrove not found, None
Skipping game 746660 due to missing pitcher stats: Pitcher Rhett Lowder not found, Pitcher Jared Jones not found
Skipping game 745768 due to missing pitcher stats: Pitcher Sean Manaea not found, None
Skipping game 746012 due to missing pitcher stats: Pitcher Adam Oller not found, None
Skipping game 745929 due to missing pitcher stats: None, Pitcher Jordan Montgomery not found
Skipping game 745521 due to missing pitcher stats: None, Pitcher Javier Assad not found
Skipping game 744795 due to missing pitcher stats: Pitcher DJ Herz not found, None
Skipping game 745691 due to missing pitcher stats: Pitcher Gerrit Cole not found, None
Skipping game 746088 due to missing pitcher stats: None, Pitcher Joe Musgrove not found
Skipping game 745692 due to missing pitcher stats: None, Pitcher Jared Jones not found
Skipping game 744879 due to missing pitcher stats: None, Pitcher Adam Oller not found
Skipping game 

[]

In [23]:
# Run a query to get all of the items in the table and convert it to a csv for accessibility
sql_query = "SELECT * FROM mlb_game_data LIMIT 2000;"
df = pd.read_sql(sql_query, db)
df.to_csv("mlb_games.csv", index=False)

C:\Users\dylgi\AppData\Local\Temp\ipykernel_24300\1769458306.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, db)
